In [ ]:
#hide
#default_exp ingest


In [ ]:
#export
import re
import datetime

import sc2reader
import pymongo
import sc2reaper.init_ingest as ings

# Python std_lib dependencies 
from typing import Generator, Tuple
from dataclasses import dataclass, asdict
from pathlib import Path
from datetime import datetime

# Import dependencies from this package
from sc2trainingg.load_config import load_configurations

# ingest

> This module contains the tools needed to process a set of StarCraft II replays. This process entails extracting various features that describe each players' performance during a 1v1 online match and organising all the information in a set of collections that compose a MongoDB database.

In [ ]:
#export
# Definition of the initial data needed to function
CONFIG_PATH = Path.cwd() / "config.json"
CONFIG = load_configurations(CONFIG_PATH)

# Define the client and data base to work with MongoDB
DB_Client = pymongo.MongoClient(CONFIG.port_address, CONFIG.port_number)
DB = DB_Client[CONFIG.db_name]
replays_info = DB['replays_info']
print(type(replays_info))
# Define the dependency the default to sc2reader
assert Path(CONFIG.replay_path).exists(), "Invalid replay path"
REPLAY_GEN = sc2reader.load_replays(CONFIG.replay_path)


# Define the inmutable dataclasses for each replay's data
@dataclass(frozen=True)
class Player_data:
    """
    Immutable dataclass that contains Information that describes a 
    player's attributes in a match.

    Attributes
    ----------
    `name: str`
        The player's user name.
    `number: int`
        Player number in the match. In a 1v1, match there would be a 
        Player 1 and 2.
    `race: str `
        The game race (Protoss, Terran, Zerg) with which the player 
        played this match.
    `result: str`
        Variable descriving whether the player was the matches winner 
        ('Win') or loser ('Loss')
    """
    name: str
    number: int
    race: str
    result: str

@dataclass(frozen=True)
class Replay_data:
    """
    Immutable dataclass that contains information summarising a 
    match's main attributes.

    Attributes
    ----------
    `replay_name: str`
        Absolute path of where the Replay was stored when uploaded.
    `replay_id: str`
        Name of the SC2Replay file.
    `date_time: datetime`
        Date and time when the match was played and recorded.
    `match_type: str`
        Descrives the team configuration of the match (eg '1v1', '2v2').
    `game_release: str`
        Version and patch number for the game release where the match
        played.
    `map_name: str`
        Name of the match's map.
    `category: str`
        Descrives if the match was 'Ladder' or other type of match.
    `winner: str`
        User name of the match's winner
    `players: Tuple[Player_data, ...]`
        Summarised information of the match's players (see Player_data 
        class).
    """
    replay_name: str
    replay_id: str
    date_time: datetime
    match_type: str
    game_release: str
    map_name: str
    category: str
    winner: str
    players: Tuple[Player_data, ...]

# Functions that format data according to the dataclasses
def extend_player_info(participant: sc2reader.objects.Participant) -> Player_data:
    '''
    Extracts the players' data from a Participant Object, into a 
    Player_data instance.

    Parameters
    ----------
    `participant: sc2reader.objects.Participant`
        Participant object containing all data related to a SC2Player

    Returns
    -------
    `Player_data`
        Summary of a player's attributes on a match.
    '''
    return Player_data(
        participant.name, 
        participant.pid, 
        participant.play_race,
        participant.result
    )

def get_replay_info(replay: sc2reader.resources.Replay) -> Replay_data:
    '''
    Replay_data dataclass instance with a replay's general 
    information.

    Parameters
    ----------
    `replay: sc2reader.resources.Replay`
        Replay object to be analysed.

    Returns
    -------
    `Replay_data`
        Summary of a matches main descriptive information. 
    '''
    file_name_regex = re.compile(r'[^\\]*[.]SC2Replay$')

    # Collect information about the match in a document. 
    return Replay_data(
        replay.filename,
        file_name_regex.search(replay.filename).group(),
        replay.start_time,
        replay.type,
        replay.release_string,
        replay.map_name,
        replay.category,
        replay.winner.players[0].name,
        tuple(extend_player_info(player) for player in tuple(replay.players))
    )

# Functions that build the Collections within the database
def not_replay_duplicate(replay: sc2reader.resources.Replay, collection_: pymongo.collection.Collection = replays_info) -> bool:
    '''
    Verify that the replay does not exist in a collection.

    Parameters
    -------------------
    `replay: sc2reader.resources.Replay`
        The replay being cheked
    `collection_: pymongo.collection.Collection`
        The collection where the existance check is being performed.

    ReturnS
    ---------------------
    `bool` 
        True if the replay is not in the collection, False if it is.
    '''
    if not collection_.count_documents(
        {'replay_name': replay.filename}, 
        limit = 1
    ):
        print(f'New replay found: {Path(replay.filename).name} \n addint to replay_info collection.')
        return True
    else:
        print(f'{replay.filename} already exists in the replay_info collection.')
        return False
    
def get_replays_data_set(rp_gen: Generator, collection_: pymongo.collection.Collection) -> Generator:
    '''
    Build a generator that can yield a group of Replay_data 
    instances that represent a the descriptive information of a set
    of replays that are found by an sc2reader replay generator
    and that have not been already added to a specific collection.

    Parameters
    ---------------------
    `rp_gen: Generator` 
        a sc2reader.resources.Replay generator that yields
        the replays found in the CONFIG.replay_path.

    `collection_: pymongo.collection.Collection` 
        the database collection that could contain the replays

    Return
    -------------------
    `Generator` 
        Yields Replay_data instances.
    '''
    return (get_replay_info(replay) for replay in rp_gen if not_replay_duplicate(replay, collection_))

def build_replay_info(
    rp_gen: Generator = REPLAY_GEN,
    db_collection:pymongo.collection.Collection = replays_info
    ) -> None:
    '''
    Triggers the search for new replays at CONFIG.replay_path. Adds the 
    information description of the replays to the a data collection within 
    a MongoDB data base, if they are not in the database already.

    Parameters
    ------------------
    `rp_gen: Generator = REPLAY_GEN`
        sc2reader.resources.Replay generator that yields the replays found 
        in the CONFIG.replay_path. 
    `db_collection:pymongo.collection.Collection = replays_info` 
        the database where the function adds the new documents.
    '''
    replays_data_set = [asdict(replay_data) for replay_data in get_replays_data_set(rp_gen, db_collection) if replay_data != None]
    if replays_data_set:
        db_collection.insert_many(replays_data_set)
    else:
        print(f'No new replays at {CONFIG.replay_path}') 

def build_reaper_collections() -> None:
    """
    Calls the ingest function on the sc2reaper package. Make sure
    you install the package from https://github.com/miguelgondu/sc2reaper
    in your environment before running.

    Raises
    ------
    `ImportError`
        If sc2reaper is not installed in the environment.

    """
    try:
        ings.ingest(CONFIG.replay_path, 4)
    except ImportError as ime:
        print("This program needs sc2reaper to be installed before running.")
        print("Check install instructions at https://github.com/miguelgondu/sc2reaper")
        raise ime
    except ValueError as vale:
        # If ing.ingest raises a ValueError asume reaper did not find new replays.
        print("No new .SC2Replays found by sc2reaper")